In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
# values = tf.io.read_file('soccer_ball.jpg')

In [5]:
values = tf.io.read_file('soc.png')

In [7]:
def decode_image(filename, image_type, resize_shape, channels):
    value = tensorflow.io.read_file(filename)
    if image_type == 'png':
        decoded_image = tensorflow.image.decode_png(value, channels=channels)
    elif image_type == 'jpeg':
        decoded_image = tensorflow.image.decode_jpeg(value, channels=channels)
    else:
        decoded_image = tensorflow.image.decode_image(value, channels=channels)
    
    if resize_shape is not None and image_type in ['png', 'jpeg']:
        decoded_image = tf.image.resize(decoded_image, resize_shape)
    
    return decoded_image

In [8]:
def get_dataset(image_paths, image_type, resize_shape, channels):
    filename_tensor = tf.constant(image_paths)
    dataset = tf.data.Dataset.from_tensor_slices(filename_tensor)
    
    def _map_fn(filename):
        decode_images = decode_image(filename, image_type, resize_shape, channels=channels)
        return decode_images
    
    map_dataset = dataset.map(_map_fn) # we use the map method: allow to apply the function _map_fn to all the 
    # elements of dataset 
    return map_dataset

In [9]:
def get_image_data(image_paths, image_type, resize_shape, channels):
    dataset = get_dataset(image_paths, image_type, resize_shape, channels)
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
    next_image = iterator.get_next()
    
    return next_image

In [17]:
get_image_data(['soc.png'],'png',(10,20),0)[3]

<tf.Tensor: shape=(20, 3), dtype=float32, numpy=
array([[255.     , 255.     , 254.93   ],
       [  0.     ,   0.     ,   0.     ],
       [251.6    , 251.6    , 251.84999],
       [252.79999, 252.36   , 252.68999],
       [  0.     ,   0.     ,   0.     ],
       [251.62999, 251.62999, 251.45999],
       [250.     , 250.     , 249.64   ],
       [239.25   , 239.2    , 238.5    ],
       [255.     , 254.96   , 254.99   ],
       [255.     , 254.93   , 255.     ],
       [255.     , 255.     , 255.     ],
       [255.     , 255.     , 255.     ],
       [255.     , 255.     , 255.     ],
       [255.     , 255.     , 255.     ],
       [255.     , 254.97   , 255.     ],
       [255.     , 255.     , 254.97   ],
       [217.74016, 230.13011,  99.3103 ],
       [228.05   , 242.34999,  79.05   ],
       [255.     , 254.23999, 254.81   ],
       [255.     , 254.87   , 255.     ]], dtype=float32)>

In [18]:
img_rows = 28 
img_cols = 28 
num_classes = 10

In [19]:
import tensorflow as tf # tensorflow 2.0
from keras.datasets import mnist
import numpy as np
seed=0
np.random.seed(seed) # fix random seed
tf.random.set_seed(seed)
# input image dimensions
num_classes = 10 # 10 digits

img_rows, img_cols = 28, 28 # number of pixels 

# the data, shuffled and split between train and test sets
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11501568/11490434 [==============================] - 2s 0us/step


In [26]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1) 
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
# cast floats to single precision
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# rescale data in interval [0,1]
X_train /= 255
X_test /= 255

In [22]:
from keras.utils import np_utils

In [24]:
from keras import utils as np_utils

In [28]:
from tensorflow import keras

In [30]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout
model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(num_classes, activation='softmax'))

In [32]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
#train the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=3)

Epoch 1/3
1875/1875 [==============================] - 38s 20ms/step - loss: 0.6793 - accuracy: 0.7728 - val_loss: 0.2075 - val_accuracy: 0.9341
Epoch 2/3
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1977 - accuracy: 0.9377 - val_loss: 0.1248 - val_accuracy: 0.9598
Epoch 3/3
1875/1875 [==============================] - 46s 24ms/step - loss: 0.1350 - accuracy: 0.9573 - val_loss: 0.0912 - val_accuracy: 0.9701


In [34]:
# evaluate the model
score = model.evaluate(X_test, Y_test, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 2s 5ms/step - loss: 0.0912 - accuracy: 0.9701

Test loss: 0.09124460816383362
Test accuracy: 0.9700999855995178


In [37]:
#predict first 4 images in the test set
preds=model.predict(X_test[:4])

In [40]:
[max(i) for i in preds]

[0.9999379, 0.99763894, 0.9993229, 0.9999156]

In [41]:
preds

array([[6.89449564e-07, 8.76362094e-08, 3.87916407e-05, 1.34091388e-05,
        3.83278895e-11, 2.50273075e-07, 5.33007552e-15, 9.99937892e-01,
        1.97781873e-07, 8.78392893e-06],
       [5.67475263e-06, 2.31380225e-03, 9.97638941e-01, 2.59048447e-05,
        5.82381018e-11, 1.73229765e-07, 3.46786948e-07, 7.94168198e-09,
        1.50830656e-05, 2.08743895e-12],
       [3.51193303e-05, 9.99322891e-01, 7.58699825e-05, 4.83870599e-06,
        1.27949112e-04, 5.99100940e-06, 1.84599732e-04, 1.09721695e-04,
        1.23761944e-04, 9.29949420e-06],
       [9.99915600e-01, 2.04428488e-10, 2.21941991e-05, 2.40472275e-07,
        3.33413546e-08, 1.65140955e-06, 1.48210165e-05, 6.77167236e-07,
        8.19372417e-07, 4.38598581e-05]], dtype=float32)

In [44]:
X_test[0]

array([[[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]

In [45]:
from IPython.utils import io
with io.capture_output() as captured:
  !git clone https://github.com/openai/CLIP
  # !pip install taming-transformers
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/mfrashad/clipit.git
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  # ClipDraw deps
  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !pip install gradio

  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  # !ls
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models

In [53]:
import sys
sys.path.append("clipit")
import clipit

# To reset settings to default
clipit.reset_settings()

# You can use "|" to separate multiple prompts
prompts = "underwater city"

# You can trade off speed for quality: draft, normal, better, best
quality = "normal"

# Aspect ratio: widescreen, square
aspect = "widescreen"

# Add settings
clipit.add_settings(prompts=prompts, quality=quality, aspect=aspect)

# Apply these settings and run
settings = clipit.apply_settings()
clipit.do_init(settings)
cliptit.do_run(settings)

RuntimeError: class '__torch__.kornia.geometry.boxes.Boxes3D' already defined.

In [ ]:
clipit.add_settings(iterations=500)

In [47]:
pip install omegaconf

  Using cached omegaconf-2.2.3-py3-none-any.whl (79 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install ftfy

  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [52]:
pip install clipit

ERROR: Could not find a version that satisfies the requirement clipit (from versions: none)
ERROR: No matching distribution found for clipit
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
